In [1]:
from datetime import datetime

import pytz
from database import Database
from sklearn.pipeline import Pipeline
from sqlalchemy.dialects.postgresql import insert
from ssrs import BookingData
from transformers import BookingDataEncoder, BookingDataReadCsv

local_tz = pytz.timezone("Asia/Dubai")

In [2]:
date_fr = (
    datetime.now(local_tz)
    .replace(hour=0, minute=0, second=0)
    .strftime("%m/%d/%Y %H:%M:%S")
)
date_to = datetime.now(local_tz).strftime("%m/%d/%Y %H:%M:%S")

In [3]:
db = Database()

reservation = db.gwg_reservation

/home/jovyan/work/database.py:34: SAWarning: Did not recognize type 'geometry' of column 'geom'
  Table(


In [4]:
pipeline = Pipeline(
    [
        ("csv_reader", BookingDataReadCsv()),
        ("data_encoder", BookingDataEncoder()),
    ]
)

In [5]:
dfs = []

for destination in ["DU", "OM"]:
    booking_data = BookingData(destination, date_fr, date_to, 3)
    get_bookings = booking_data.get()

    df = pipeline.fit_transform(get_bookings)

    if df is not None:
        df.to_csv(f"{destination}_OUT.csv", index=False)
        stmt = insert(reservation).values(df.to_dict(orient="records"))
        stmt = stmt.on_conflict_do_update(
            constraint="gwg_reservation_pkey", set_=dict(stmt.excluded)
        )

        with db.engine.begin() as connection:
            connection.execute(stmt)